In [5]:
# https://github.com/pytorch/examples/blob/master/mnist/main.py
from __future__ import print_function
%matplotlib inline
import pandas as pd
import random
import matplotlib.pyplot as plt
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import numpy as np
import PIL
from skimage.transform import resize
from random import shuffle
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [6]:
sites = [
    {"id": "1-800-flowers", "count" : 15,"label" : 0},
    {"id": "Alfred Sung", "count" : 15,"label" : 0},
    {"id": "Alibaba", "count" : 15,"label" : 1},
    {"id": "Amazon", "count" : 18,"label" : 1},
    {"id": "American Apparel", "count" : 15,"label" : 0},
    {"id": "Arngren", "count" : 15,"label" : 0},
    {"id": "Azteca Soccer", "count" : 15,"label" : 1},
    {"id": "Best Electronic", "count" : 17,"label" : 0},
    {"id": "BestBuy", "count" : 18,"label" : 1},
    {"id": "BigCommerce", "count" : 18,"label" : 1},
    {"id": "Blinkee", "count" : 16,"label" : 0},
    {"id": "Bliss", "count" : 15,"label" : 1},
    {"id": "BonBonBon", "count" : 19,"label" : 1},
    {"id": "Costco", "count" : 15,"label" : 0},
    {"id": "CruiseMaster", "count" : 15,"label" : 1},
    {"id": "Custom Barres", "count" : 15,"label" : 1},
    {"id": "CVS", "count" : 15,"label" : 0},
    {"id": "Decibullz", "count" : 15,"label" : 1},
    {"id": "DSW", "count" : 16,"label" : 0},
    {"id": "eBay", "count" : 16,"label" : 1},
    {"id": "Electrifying Times", "count" : 15,"label" : 0},
    {"id": "Fanatics", "count" : 15,"label" : 0},
    {"id": "Flipkart", "count" : 15,"label" : 1},
    {"id": "Footlocker", "count" : 15,"label" : 0},
    {"id": "Free City Supershop", "count" : 11,"label" : 0},
    {"id": "H & M", "count" : 19,"label" : 0},
    {"id": "Home Science Tools", "count" : 15,"label" : 1},
    {"id": "Ikea", "count" : 16,"label" : 0},
    {"id": "Jabong", "count" : 15,"label" : 1},
    {"id": "Jeep People", "count" : 15,"label" : 1},
    {"id": "Koi Computer", "count" : 15,"label" : 1},
    {"id": "Lazada", "count" : 16,"label" : 1},
    {"id": "Ling's Cars", "count" : 15,"label" : 0},
    {"id": "Mednat", "count" : 15,"label" : 0},
    {"id": "Mercia Tourist Board", "count" : 12,"label" : 0},
    {"id": "Microcenter.com", "count" : 18, "label" : 0},
    {"id": "Next Chapter", "count" : 15,"label" : 1},
    {"id": "Nordstrom", "count" : 16,"label" : 0},
    {"id": "Opensky", "count" : 15,"label" : 0},
    {"id": "P & M Computers", "count" : 15,"label" : 0},
    {"id": "Paradise With A View", "count" : 15,"label" : 0},
    {"id": "Pennyjuice", "count" : 12,"label" : 0},
    {"id": "Rakuten", "count" : 19,"label" : 0},
    {"id": "RalphLauren.com", "count" : 19,"label" : 0},
    {"id": "Renogy", "count" : 16,"label" : 1},
    {"id": "Rusty Surfboards", "count" : 15,"label" : 1},
    {"id": "Shopee", "count" : 16,"label" : 1},
    {"id": "Shopify", "count" : 15,"label" : 1},
    {"id": "Signal Boosters", "count" : 15,"label" : 1},
    {"id": "Skullcandy", "count" : 16,"label" : 1},
    {"id": "SnapDeal", "count" : 15,"label" : 1},
    {"id": "Suzanne Collins Books", "count" : 15,"label" : 0},
    {"id": "The Mountain", "count" : 18,"label" : 1},
    {"id": "True Links Wear", "count" : 16,"label" : 1},
    {"id": "Urban Outfitters", "count" : 15,"label" : 0},
    {"id": "Walgreens", "count" : 16,"label" : 0},
    {"id": "Walmart", "count" : 15,"label" : 1},
    {"id": "Water Equipment", "count" : 18,"label" : 0},
    {"id": "Woocommerce", "count" : 15,"label" : 1},
    {"id": "Zalora", "count" : 16,"label" : 1}   
]
ctr0 = 0
ctr1 = 0
for s in sites:
    if s["label"] == 0:
        ctr0 += s["count"]
    else:
        ctr1 += s["count"]
print(ctr0 * 0.05, ctr1 * 0.05)

23.25 23.650000000000002


In [8]:
folds = [[] for i in range(5)]
good = [x for x in sites if x["label"] == 1]
bad = [x for x in sites if x["label"] == 0]
shuffle(good)
shuffle(bad)
print(len(good))
print(len(bad))
for i in range(30):
    folds[i % 5].append(good[i])
    folds[i % 5].append(bad[i])

for i in range(5):
    print("Fold",i + 1);
    for j in folds[i]:
        print(j["id"],j["count"],j["label"])
    print()

30
30
Fold 1
Rusty Surfboards 15 1
Ling's Cars 15 0
Shopify 15 1
Opensky 15 0
Shopee 16 1
Free City Supershop 11 0
Jabong 15 1
Microcenter.com 18 0
Alibaba 15 1
Ikea 16 0
Azteca Soccer 15 1
Water Equipment 18 0

Fold 2
Zalora 16 1
Mercia Tourist Board 12 0
The Mountain 18 1
1-800-flowers 15 0
Walmart 15 1
Walgreens 16 0
CruiseMaster 15 1
Electrifying Times 15 0
SnapDeal 15 1
Blinkee 16 0
BestBuy 18 1
DSW 16 0

Fold 3
Amazon 18 1
American Apparel 15 0
Skullcandy 16 1
Costco 15 0
Koi Computer 15 1
H & M 19 0
Jeep People 15 1
Best Electronic 17 0
eBay 16 1
Arngren 15 0
Decibullz 15 1
Fanatics 15 0

Fold 4
Next Chapter 15 1
Rakuten 19 0
Lazada 16 1
Paradise With A View 15 0
True Links Wear 16 1
Alfred Sung 15 0
BigCommerce 18 1
Pennyjuice 12 0
Signal Boosters 15 1
Urban Outfitters 15 0
Flipkart 15 1
P & M Computers 15 0

Fold 5
Custom Barres 15 1
RalphLauren.com 19 0
BonBonBon 19 1
Footlocker 15 0
Renogy 16 1
CVS 15 0
Bliss 15 1
Suzanne Collins Books 15 0
Home Science Tools 15 1
Mednat 15 

In [10]:
data = [[] for i in range(5)]
labels = [[] for i in range(5)]
dataVal = []
labelsVal = []
sum = 0
for i in range(0,5):
    val = 0
    for site in folds[i]:
        print("Loading {}".format(site["id"]))
        sum += site["count"]
        trig = random.randint(1,site['count'])
        for j in range(1,site['count'] + 1):
            if j == trig and val % 5 != 0:
                labelsVal = labelsVal + [site["label"]]
                temp = PIL.Image.open("%s/%s - %02d.png" % (site['id'],site['id'],j))
                temp = np.array(temp);
                temp = resize(temp, (temp.shape[0] // 2, temp.shape[1] // 2), anti_aliasing=True)
                dataVal.append(temp)
#                 print("Loading %s/%s - %02d.png" % (site['id'],site['id'],j))
            else:
                labels[i] = labels[i] + [site["label"]]
                temp = PIL.Image.open("%s/%s - %02d.png" % (site['id'],site['id'],j))
                temp = np.array(temp);
                temp = resize(temp, (temp.shape[0] // 2, temp.shape[1] // 2), anti_aliasing=True)
                data[i].append(temp)
#                 print("Loading %s/%s - %02d.png" % (site['id'],site['id'],j))
        val += 1
print("Loaded %d screens." % sum)
print("Validation Samples: {}".format(len(dataVal)))
# print(labels)

Loading Rusty Surfboards


C:\ProgramData\Anaconda3\lib\site-packages\skimage\transform\_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Loading Ling's Cars
Loading Shopify
Loading Opensky
Loading Shopee
Loading Free City Supershop
Loading Jabong
Loading Microcenter.com
Loading Alibaba
Loading Ikea
Loading Azteca Soccer
Loading Water Equipment
Loading Zalora
Loading Mercia Tourist Board
Loading The Mountain
Loading 1-800-flowers
Loading Walmart
Loading Walgreens
Loading CruiseMaster
Loading Electrifying Times
Loading SnapDeal
Loading Blinkee
Loading BestBuy
Loading DSW
Loading Amazon
Loading American Apparel
Loading Skullcandy
Loading Costco
Loading Koi Computer
Loading H & M
Loading Jeep People
Loading Best Electronic
Loading eBay
Loading Arngren
Loading Decibullz
Loading Fanatics
Loading Next Chapter
Loading Rakuten
Loading Lazada
Loading Paradise With A View
Loading True Links Wear
Loading Alfred Sung
Loading BigCommerce
Loading Pennyjuice
Loading Signal Boosters
Loading Urban Outfitters
Loading Flipkart
Loading P & M Computers
Loading Custom Barres
Loading RalphLauren.com
Loading BonBonBon
Loading Footlocker
Loading

In [12]:
for j in range(len(data)):
    data[j] = np.transpose(data[j],(0,3,1,2))
    print(np.array(data[j]).shape)
dataVal = np.transpose(dataVal,(0,3,1,2))
print(np.array(dataVal).shape)

(45, 3, 384, 683)


In [14]:
import pickle

with open('dataconvall.pt', 'wb') as f:
    pickle.dump(data, f)
    
with open('labelsall.pt', 'wb') as f:
    pickle.dump(labels, f)
    
with open('datavalconvall.pt', 'wb') as f:
    pickle.dump(dataVal, f)
    
with open('labelsval.pt', 'wb') as f:
    pickle.dump(labelsVal, f)